In [1]:
import pandas as pd

In [2]:
data = pd.read_pickle("preparedData.pkl")
data

,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,userID,combinedSpaceStationID,timeConnected,idleTime,chargingTime,weekday,month,time,WhPerMile,kWhRequested,milesRequested,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture
0,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1,194.0,AG-3F30/1-1-179-810,21741.0,5980.0,15761.0,3,1,13:08:54,250.0,25.00,100.0,463.0,"Thu, 02 Jan 2020 13:09:39 GMT",True,"Thu, 02 Jan 2020 20:51:54 GMT"
1,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1,4275.0,AG-1F01/1-1-193-825,32491.0,8416.0,24075.0,3,1,13:36:50,280.0,70.00,250.0,595.0,"Thu, 02 Jan 2020 13:37:11 GMT",True,"Thu, 02 Jan 2020 23:31:50 GMT"
2,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1,344.0,AG-1F03/1-1-193-829,38567.0,29056.0,9511.0,3,1,13:56:35,400.0,8.00,20.0,60.0,"Thu, 02 Jan 2020 13:57:17 GMT",True,"Thu, 02 Jan 2020 14:56:35 GMT"
3,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1,1117.0,AG-1F04/1-1-193-820,9521.0,4794.0,4727.0,3,1,13:59:58,400.0,8.00,20.0,65.0,"Thu, 02 Jan 2020 14:00:03 GMT",True,"Thu, 02 Jan 2020 15:04:58 GMT"
4,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1,334.0,AG-1F06/1-1-193-819,29319.0,13870.0,15449.0,3,1,14:00:01,400.0,16.00,40.0,504.0,"Thu, 02 Jan 2020 14:00:13 GMT",True,"Thu, 02 Jan 2020 22:24:01 GMT"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56531,2019-07-01 19:13:11+00:00,2019-07-02 00:44:40+00:00,2019-07-01 21:13:56+00:00,6.048,1,2643.0,AG-3F23/1-1-179-797,19889.0,12644.0,7245.0,0,7,19:13:11,275.0,8.25,30.0,374.0,"Mon, 01 Jul 2019 19:13:10 GMT",True,"Tue, 02 Jul 2019 01:27:11 GMT"
56532,2019-07-01 19:43:46+00:00,2019-07-02 02:58:53+00:00,2019-07-02 02:43:53+00:00,41.047,1,394.0,AG-4F38/1-1-191-793,26107.0,900.0,25207.0,0,7,19:43:46,231.0,36.96,160.0,310.0,"Mon, 01 Jul 2019 19:43:58 GMT",True,"Tue, 02 Jul 2019 00:53:46 GMT"
56533,2019-07-01 20:34:30+00:00,2019-07-02 00:30:52+00:00,2019-07-01 21:27:44+00:00,5.390,1,2276.0,AG-3F15/1-1-179-813,14182.0,10988.0,3194.0,0,7,20:34:30,300.0,12.00,40.0,381.0,"Mon, 01 Jul 2019 20:34:29 GMT",True,"Tue, 02 Jul 2019 02:55:30 GMT"
56534,2019-07-01 21:49:12+00:00,2019-07-02 01:37:18+00:00,2019-07-01 23:14:33+00:00,8.399,1,1346.0,AG-3F18/1-1-179-798,13686.0,8565.0,5121.0,0,7,21:49:12,400.0,32.00,80.0,36.0,"Mon, 01 Jul 2019 21:49:37 GMT",True,"Mon, 01 Jul 2019 22:25:12 GMT"


### KPI1
Station Downtime Between Sessions
total accumulated disconnect time
Description: Measures the duration of time that a charging station remains "free" or inactive before the next connection occurs, indicating how quickly the station is utilized between sessions

In [5]:
# Sortieren Sie das DataFrame nach 'combinedSpaceStationID' und 'disconnectTime'
data = data.sort_values(by=['combinedSpaceStationID', 'disconnectTime'])

# Berechnen Sie die Downzeiten und speichern Sie sie in einer neuen Spalte 'downTime'
data['downTime'] = data.groupby('combinedSpaceStationID')['connectionTime'].shift(-1) - data['disconnectTime']

# Füllen Sie NaN-Werte in der letzten Zeile der Gruppen mit 0 (da es keine nächste Session gibt)
data['downTime'] = data['downTime'].fillna(pd.Timedelta(seconds=0))

# Summieren Sie die Downzeiten und berechnen Sie den Durchschnitt für jede 'combinedSpaceStationID'
result = data.groupby('combinedSpaceStationID')['downTime'].agg(['sum', 'mean']).reset_index()
result.columns = ['combinedSpaceStationID', 'sumDownTime', 'avgDownTime']

result

,combinedSpaceStationID,sumDownTime,avgDownTime
0,AG-1F01/1-1-193-825,859 days 02:42:33,1 days 02:40:25.036222509
1,AG-1F02/1-1-193-827,816 days 06:55:40,0 days 21:17:40.152173913
2,AG-1F03/1-1-193-829,864 days 01:02:15,1 days 03:27:58.589403973
3,AG-1F04/1-1-193-820,820 days 15:00:03,0 days 19:54:50.599595551
4,AG-1F05/1-1-193-816,859 days 01:11:27,1 days 03:44:54.868102288
...,...,...,...
101,CA-509/2-39-129-563,600 days 23:51:03,7 days 10:04:20.037037037
102,CA-510/2-39-130-564,627 days 00:41:01,4 days 17:08:52.789473684
103,CA-511/2-39-131-565,554 days 05:03:38,5 days 14:21:14.929292929
104,CA-512/2-39-138-566,1138 days 04:18:11,6 days 20:33:21.753012048


### KPI 2.0: Total Idle Time
Sums up the idle time per combinedSpaceStationID (in seconds)
Formula: Sum of idle time per combinedSpaceStationID

### KPI 2.1: Mean Idle Time
Calculates the mean idle time per combinedSpaceStationID (in seconds)
Formula: Mean idle time per combinedSpaceStationID

In [6]:
result_idle_sum = data.groupby('combinedSpaceStationID')['idleTime'].sum()

result_idle_mean = data.groupby('combinedSpaceStationID')['idleTime'].mean()

print(result_idle_sum)
print(result_idle_mean)

combinedSpaceStationID
AG-1F01/1-1-193-825     7370035.0
AG-1F02/1-1-193-827    11088828.0
AG-1F03/1-1-193-829     7055863.0
AG-1F04/1-1-193-820     9944936.0
AG-1F05/1-1-193-816     6982929.0
                          ...    
CA-509/2-39-129-563      845806.0
CA-510/2-39-130-564     1112803.0
CA-511/2-39-131-565     1060237.0
CA-512/2-39-138-566     1656945.0
CA-513/2-39-139-567      894336.0
Name: idleTime, Length: 106, dtype: float64
combinedSpaceStationID
AG-1F01/1-1-193-825     9534.327296
AG-1F02/1-1-193-827    12053.073913
AG-1F03/1-1-193-829     9345.513907
AG-1F04/1-1-193-820    10055.547017
AG-1F05/1-1-193-816     9398.289367
                           ...     
CA-509/2-39-129-563    10442.049383
CA-510/2-39-130-564     8366.939850
CA-511/2-39-131-565    10709.464646
CA-512/2-39-138-566     9981.596386
CA-513/2-39-139-567     8599.384615
Name: idleTime, Length: 106, dtype: float64


### KPI 3: Station Utilization
Quantifies the relation between the total time of usage of a station and the total elapsed time since the first session record
Formula: (Total Sum of timeConnected) / total hours

In [7]:
first_record = data['connectionTime'].min()
last_record = data['connectionTime'].max()
#today_date = pd.Timestamp.now(tz = 'Europe/Berlin')
#range = today_date - first_record
range = last_record - first_record

station_utilization = data.groupby('combinedSpaceStationID')['timeConnected'].sum()

station_utilization = pd.to_timedelta(station_utilization, unit="s") / range

station_utilization

#1 Strings userID
#2 Zeiten als Minuten oder anderes Format
#Ratio Sinnvoller?!?!?
#may change over time????

combinedSpaceStationID
AG-1F01/1-1-193-825    0.171352
AG-1F02/1-1-193-827    0.205730
AG-1F03/1-1-193-829    0.167178
AG-1F04/1-1-193-820    0.202516
AG-1F05/1-1-193-816    0.171560
                         ...   
CA-509/2-39-129-563    0.017247
CA-510/2-39-130-564    0.026828
CA-511/2-39-131-565    0.019941
CA-512/2-39-138-566    0.036668
CA-513/2-39-139-567    0.019579
Name: timeConnected, Length: 106, dtype: float64

### KPI4
Most Frequently Utilized Stations
Description: Identifies the stations that are used most frequently based on customer session data

In [9]:
#Group data by 'station_id' and count the number of occurrences
station_usage_counts = data['combinedSpaceStationID'].value_counts()

#Sort the stations by usage count in descending order
sorted_stations = station_usage_counts.sort_values(ascending=False)

#Display the most frequently used stations
print("Most Frequently Utilized Stations:")
print(sorted_stations.head(10))  # Adjust the number to display more or fewer stations

Most Frequently Utilized Stations:
combinedSpaceStationID
CA-303/2-39-139-28     1696
CA-305/2-39-131-30     1330
AG-1F08/1-1-178-823    1217
AG-1F10/1-1-178-828    1077
CA-307/2-39-129-17     1061
CA-315/2-39-89-25      1024
AG-1F06/1-1-193-819    1019
AG-1F04/1-1-193-820     989
CA-311/2-39-125-21      989
CA-309/2-39-127-19      970
Name: count, dtype: int64


### KPI 5: Site utilization
Counts the number of session for each site
Formula: Count of entries for each siteID

In [11]:
site_usage_counts = data['siteID'].value_counts()

sorted_sites = site_usage_counts.sort_values(ascending=False)

print("Most Frequently Utilized Sites:")
print(sorted_sites)

Most Frequently Utilized Sites:
siteID
1    28384
2    28152
Name: count, dtype: int64


### KPI 6.0: Total kWh Charged per station
Sum up energy consumption for each station
Formula: Sum of energy consumption for each station

### KPI 6.1: Total kWh Charged (in the last 24 hours) per station
Sum up energy consumption for each station in last 24 hours
Formula: Sum of energy consumption for each station in last 24 hours

In [12]:
total_energy_consumption = data.groupby('combinedSpaceStationID')['kWhDelivered'].sum()

timestamp_now = pd.Timestamp.now(tz = 'Europe/Berlin')
total_energy_consumption_recent = data[timestamp_now - data['connectionTime'] <= timedelta(hours=24)].groupby('combinedSpaceStationID')['kWhDelivered'].sum()

print(total_energy_consumption)
print(total_energy_consumption_recent)

NameError: name 'charging_sessions_modified' is not defined

### KPI7
Average Energy Delivered per Session
Formula: (Total kWh Charged) / (Total Number of Sessions)
Description: Calculates the average energy consumption per charging session, aiding in assessing energy delivery efficiency

In [14]:
overall_average_energy_delivered = data['kWhDelivered'].mean()
overall_average_energy_delivered

11.381496936069817

### KPI8
Average Session Duration
Formula: (Total Session Duration) / (Total Number of Sessions)
Description: Measures the average duration of charging sessions, assisting in evaluating station turnover rates

In [15]:
avg_session_duration = data['timeConnected'].mean()

# Anzeigen oder Verwenden der durchschnittlichen Dauer einer Session
print("Durchschnittliche Dauer einer Session:", avg_session_duration)

Durchschnittliche Dauer einer Session: 23524.884639875476


### KPI9
Peak Demand Hours
Description: Identifies the hours with the highest energy demand, facilitating load management and pricing optimization

### KPI10
Registered and Unregistered Customer Ratio
Formula: (Number of Registered Customers) / (Number of Unregistered Customers)
Description: Compares the ratio of registered (loyal) customers to unregistered (casual) customers, providing insights into registration program

### KPI 11: Repeating Customer Rate
Expresses the percentage of customers who return for multiple charging sessions, indicating customer loyalty
Formula: Number of repeating customers / total number of customers

In [17]:
number_of_session = len(data)
number_of_sessions_with_uf = len(data[data['userID'].notna()])

number_customer_total = data['userID'].dropna().astype(str).nunique()

returning_customer = data.groupby('userID').size()
returning_customer = returning_customer[returning_customer > 1].count()


print(
    f"With {number_of_sessions_with_uf/number_of_session:.2f} percent of the charging sessions we can identify the user based on the userID. "
    f"From the {number_customer_total} customers there are {returning_customer} returning users. "
    f"The ratio between returning customers and the total number of customers is {returning_customer/number_customer_total:.2f} percent."
)

With 0.73 percent of the charging sessions we can identify the user based on the userID. From the 872 customers there are 743 returning users. The ratio between returning customers and the total number of customers is 0.85 percent.
